In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import model_evaluation_functions

In [ ]:
%load_ext autoreload
%autoreload 2

# Load and clean prediciton data

In [ ]:
df_ground_truth = pd.read_csv('../annotations/ground_truth_boat_frames_coco.csv', index_col=0, sep=';')
df_yolo_predictions = pd.read_csv('../annotations/yolov8x_no_train_labels.csv', index_col=0, sep=';')

df_ground_truth['datetime'] = pd.to_datetime(df_ground_truth.datetime)
df_ground_truth['date'] = pd.to_datetime(df_ground_truth.datetime.dt.date, format='%Y-%m-%d')
df_yolo_predictions['datetime'] = pd.to_datetime(df_yolo_predictions['datetime'])
df_yolo_predictions['date'] = pd.to_datetime(df_yolo_predictions.datetime.dt.date, format='%Y-%m-%d')
print('shape of loaded data', df_ground_truth.shape, df_yolo_predictions.shape)
print('Ground truth data:')
print(df_ground_truth.groupby(['date', 'camera_id']).datetime.count())
print('YOLO predictions data:')
print(df_yolo_predictions.groupby(['date', 'camera_id']).datetime.count())

# filter out predictions for dates with ground truth data
df_yolo_predictions = df_yolo_predictions[\
    ((df_yolo_predictions.date == '2023-06-09') & (df_yolo_predictions.camera_id.isin([1,2]))) |\
    ((df_yolo_predictions.date == '2023-06-10') & (df_yolo_predictions.camera_id == 2)) |\
    ((df_yolo_predictions.date == '2023-07-07') & (df_yolo_predictions.camera_id == 2)) |\
    ((df_yolo_predictions.date == '2023-07-08') & (df_yolo_predictions.camera_id == 1)) \
].copy()

# crop bounding boxes from right side of camera 2 field of view
df_yolo_predictions.drop(index=df_yolo_predictions[(df_yolo_predictions.camera_id == 2) & (df_yolo_predictions.x > 1800)].index, inplace=True)
df_ground_truth.drop(index=df_ground_truth[(df_ground_truth.camera_id == 2) & (df_ground_truth.x > 1800)].index, inplace=True)

df_ground_truth.set_index('filename', inplace=True)
df_yolo_predictions.set_index('filename', inplace=True)

df_ground_truth.shape, df_yolo_predictions.shape

In [ ]:
# transform coords to xmin, ymin, xmax, ymax
df_ground_truth['xmin'] = (df_ground_truth['x'] - df_ground_truth['w']/2).astype(int)
df_ground_truth['ymin'] = (df_ground_truth['y'] - df_ground_truth['h']/2).astype(int)
df_ground_truth['xmax'] = (df_ground_truth['x'] + df_ground_truth['w']/2).astype(int)
df_ground_truth['ymax'] = (df_ground_truth['y'] + df_ground_truth['h']/2).astype(int)
df_yolo_predictions['xmin'] = (df_yolo_predictions['x'] - df_yolo_predictions['w']/2).astype(int)
df_yolo_predictions['ymin'] = (df_yolo_predictions['y'] - df_yolo_predictions['h']/2).astype(int)
df_yolo_predictions['xmax'] = (df_yolo_predictions['x'] + df_yolo_predictions['w']/2).astype(int)
df_yolo_predictions['ymax'] = (df_yolo_predictions['y'] + df_yolo_predictions['h']/2).astype(int)

In [ ]:
# filter out predicitons in the bank on camera 1; use a line to define the bank in the camera 1 field of view
point1_cam01_bank = (489, 591)
point2_cam01_bank = (1499, 875)
slope_cam01_bank = (point2_cam01_bank[1] - point1_cam01_bank[1]) / (point2_cam01_bank[0] - point1_cam01_bank[0])
intercept_cam01_bank = point1_cam01_bank[1] - slope_cam01_bank * point1_cam01_bank[0]

def under_the_bank_apply(row):
    if row.camera_id == 1 and row.x >= point1_cam01_bank[0] and row.x <= point2_cam01_bank[0]:
        if row.y > (slope_cam01_bank * row.x + intercept_cam01_bank):
            return True        
    return False
        
df_yolo_predictions['under_the_bank'] = df_yolo_predictions.apply(under_the_bank_apply, axis=1)
print('How many prediction were under the bank?')
print(df_yolo_predictions['under_the_bank'].value_counts())
df_yolo_predictions = df_yolo_predictions[df_yolo_predictions.under_the_bank == False].copy()
print('YOLO predictions filtered data:')
print(df_yolo_predictions.groupby(['date', 'camera_id']).datetime.count())

# Evaluation of precision and recall for detected frame_ids

In [ ]:
df_evaluation, total_evaluation_dict = model_evaluation_functions.evaluate_model_frame_ids_score(df_ground_truth, df_yolo_predictions)
total_evaluation_dict

In [ ]:
# df_evaluation.to_csv('data_evaluation_zero_shot.csv', sep=';')

In [ ]:
model_evaluation_functions.evaluate_mAP(df_ground_truth, df_yolo_predictions)

## Use validation part of data

In [ ]:
df_ground_truth_filtered = df_ground_truth[df_ground_truth.date.isin(['2023-07-07', '2023-07-08'])]
df_yolo_predictions_filtered = df_yolo_predictions[df_yolo_predictions.date.isin(['2023-07-07', '2023-07-08'])]
df_ground_truth_filtered.shape, df_yolo_predictions_filtered.shape

In [ ]:
df_evaluation, total_evaluation_dict = model_evaluation_functions.evaluate_model_frame_ids_score(df_ground_truth_filtered, df_yolo_predictions_filtered)
total_evaluation_dict

In [ ]:
model_evaluation_functions.evaluate_mAP(df_ground_truth_filtered, df_yolo_predictions_filtered)

## Use validation part of data and only a filter images with known ground truth

In [ ]:
df_ground_truth_filtered = df_ground_truth[(df_ground_truth.date == '2023-07-07') | (df_ground_truth.date == '2023-07-08')].reset_index().set_index(['filename', 'frame_id'])
df_yolo_predictions_filtered = df_yolo_predictions.reset_index().set_index(['filename', 'frame_id'])
df_yolo_predictions_filtered = df_yolo_predictions_filtered.loc[df_ground_truth_filtered.index.intersection(df_yolo_predictions_filtered.index)]
df_ground_truth_filtered.shape, df_yolo_predictions_filtered.shape

In [ ]:
df_ground_truth_filtered = df_ground_truth_filtered.reset_index().set_index('filename')
df_yolo_predictions_filtered = df_yolo_predictions_filtered.reset_index().set_index('filename')
df_evaluation, total_evaluation_dict = model_evaluation_functions.evaluate_model_frame_ids_score(df_ground_truth_filtered, df_yolo_predictions_filtered)
total_evaluation_dict

In [ ]:
model_evaluation_functions.evaluate_mAP(df_ground_truth_filtered, df_yolo_predictions_filtered)

# Confidence values evaluation

In [ ]:
confidence_threshold_evaluation_dict = dict()
for confidence_threshold in [0.25, 0.5, 0.75, 0.9]:
    print('Confidence threshold', confidence_threshold)
    df_evaluation, total_evaluation_dict = model_evaluation_functions.evaluate_model_frame_ids_score(df_ground_truth, df_yolo_predictions[df_yolo_predictions.confidence >= confidence_threshold])    
    print(total_evaluation_dict)
    # df_evaluation.to_csv(f'data_evaluation_conf{confidence_threshold}.csv', sep=';')
    confidence_threshold_evaluation_dict[confidence_threshold] = total_evaluation_dict

In [ ]:
plt.plot(confidence_threshold_evaluation_dict.keys(), [total_evaluation_dict['recall'] for total_evaluation_dict in confidence_threshold_evaluation_dict.values()], 'x-')
plt.xlabel('Confidence threshold')
plt.ylabel('Recall')

In [ ]:
plt.plot(confidence_threshold_evaluation_dict.keys(), [total_evaluation_dict['f1'] for total_evaluation_dict in confidence_threshold_evaluation_dict.values()], 'x-')
plt.xlabel('Confidence threshold')
plt.ylabel('F1')

In [ ]:
plt.plot([total_evaluation_dict['recall'] for total_evaluation_dict in confidence_threshold_evaluation_dict.values()], [total_evaluation_dict['precision'] for total_evaluation_dict in confidence_threshold_evaluation_dict.values()], 'x-')
plt.xlabel('Recall')
plt.ylabel('Precision')

In [ ]:
plt.plot(confidence_threshold_evaluation_dict.keys(), [total_evaluation_dict['iou'] for total_evaluation_dict in confidence_threshold_evaluation_dict.values()], 'x-')
plt.xlabel('Confidence threshold')
plt.ylabel('IOU')

# Helpers for debugging

In [ ]:
stop_run_all_exec()

In [ ]:
## cfg_raw_cam_02_fhd_h265_20230609T173000 # there is a car identified as a boat outside of the field of view

In [ ]:
def iqr(x):
    return np.percentile(x, 75) - np.percentile(x, 25)
# filter out static bounding box predictions; use iqr < 100 as a threshold and later shift of x coords are less than 10
df_tmp = df_yolo_predictions[df_yolo_predictions.camera_id == 2].reset_index()
df_tmp = df_tmp.groupby('filename').datetime.count()
df_tmp = df_tmp[df_tmp > 100]
df_tmp = df_yolo_predictions.loc[df_tmp.index].reset_index().groupby('filename').agg({'x': iqr, 'y': iqr})
df_static_indexes = df_tmp[(df_tmp.x < 100) & (df_tmp.y < 100)].index

# for each static filename, calculate the shift of x coords between frames and filter out those with shift > 10
for filename in list(df_static_indexes)[5:]:
    print(filename)
    df_tmp = df_yolo_predictions.loc[filename].groupby('frame_id').x.max().reset_index()
    # df_tmp = 
    df_tmp['x_shift'] = df_tmp['x'].diff().abs()
    print(df_tmp)
    # df_tmp = df_tmp[df_tmp['x_shift'] > 10]
    # print(df_tmp)
    # df_static_indexes = df_static_indexes.drop(filename)
    # df_static_indexes = df_static_indexes.append(df_tmp.index)

    break



# .agg({'x': lambda x: np.percentile(x, 75) - np.percentile(x, 25)})

In [ ]:
# values for camera 2 bank identification.. this should be the water.. camera have a good horizontal angle, therefore check that bottom coord of bounding box is in upper area of the image y < less than straight line
x = 2 (0.001042), y = 378 (0.350000)
x = 1918 (0.998958), y = 445 (0.412037)

In [ ]:
df_tmp.set_index('frame_id').to_csv('debug.csv')

In [ ]:
df_tmp.loc['cfg_raw_cam_02_fhd_h265_20230707T202002.mkv']

In [ ]:
(df_yolo_predictions.loc['cfg_raw_cam_02_fhd_h265_20230707T202002.mkv'].groupby('frame_id').x.max() - df_yolo_predictions.loc['cfg_raw_cam_02_fhd_h265_20230707T202002.mkv'].groupby('frame_id').x.max().shift()).rolling(10).mean().plot()

In [ ]:
1597  /4/60

In [ ]:
df_yolo_predictions.loc['cfg_raw_cam_02_fhd_h265_20230707T202002.mkv'][df_yolo_predictions.loc['cfg_raw_cam_02_fhd_h265_20230707T202002.mkv'].frame_id > 2250].sort_values('frame_id').head(30)

In [ ]:
df_yolo_predictions.loc['cfg_raw_cam_02_fhd_h265_20230707T202002.mkv'].groupby('frame_id').x.max().sort_index().plot()

In [ ]:
df_yolo_predictions.loc['cfg_raw_cam_02_fhd_h265_20230707T174000.mkv'].y.describe()


In [ ]:
df_yolo_predictions.loc['cfg_raw_cam_02_fhd_h265_20230707T174000.mkv'].x.value_counts()

In [ ]:
plt.scatter(df_evaluation.iloc[0].frames_iou.keys(), df_evaluation.iloc[0].frames_iou.values())

In [ ]:
df_ground_truth.loc['cfg_raw_cam_01_fhd_h265_20230609T050002.mkv'].sort_values('frame_id')

In [ ]:
df_ground_truth.loc['cfg_raw_cam_02_fhd_h265_20230707T124001.mkv'].sort_values('frame_id').frame_id.plot()

In [ ]:
convert_frame_id = 1087
convert_frame_id / 4 / 60, convert_frame_id / 4 // 60, convert_frame_id / 4 % 60

In [ ]:
df_tmp = df_yolo_predictions.loc['cfg_raw_cam_02_fhd_h265_20230707T124001.mkv']
df_tmp[df_tmp.x < 1800].sort_values('frame_id').frame_id.plot()